# 1. Define Database

Tôi sẽ giúp bạn thêm tính năng lưu điểm số và feedback vào database.

1. Đầu tiên thêm API endpoint ở backend (api_database_app.py):

```python
@app.route('/save_evaluation', methods=['POST'])
def save_evaluation():
    try:
        data = request.json
        
        # Kết nối database
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Thêm bảng evaluations nếu chưa tồn tại
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS evaluations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                video_url TEXT,
                criteria TEXT,
                score INTEGER,
                feedback TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Insert dữ liệu đánh giá
        cursor.execute('''
            INSERT INTO evaluations (video_url, criteria, score, feedback)
            VALUES (?, ?, ?, ?)
        ''', (data['videoUrl'], data['criteria'], data['score'], data['feedback']))
        
        conn.commit()
        
        return jsonify({'message': 'Evaluation saved successfully'}), 200

    except Exception as e:
        print(f"Error saving evaluation: {e}")
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()
```

2. Sửa đổi code frontend (scripts.js) để gọi API lưu đánh giá:

```javascript
// Sửa lại event listener cho nút Save
document.getElementById('saveScoreButton').addEventListener('click', async () => {
    const scoreInput = document.getElementById('scoreInput').value;
    const noteInput = document.getElementById('noteInput').value;
    const videoUrl = document.getElementById('videoLink').value.trim();

    try {
        // Gọi API để lưu đánh giá
        const response = await fetch('http://localhost:3000/save_evaluation', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({
                videoUrl: videoUrl,
                criteria: currentCriteria,
                score: parseInt(scoreInput),
                feedback: noteInput
            })
        });

        const result = await response.json();

        if (response.ok) {
            // Hiển thị kết quả vào bảng UI
            displayResults(currentCriteria, noteInput, scoreInput);

            // Clear input fields
            document.getElementById('scoreInput').value = '';
            document.getElementById('noteInput').value = '';

            // Thông báo thành công
            alert('Evaluation saved successfully!');
        } else {
            throw new Error(result.error);
        }
    } catch (error) {
        console.error('Error saving evaluation:', error);
        alert('Error saving evaluation. Please try again.');
    }
});
```

Giải thích luồng xử lý:

1. Frontend:
- Khi user click nút Save
- Thu thập dữ liệu từ form (điểm số, feedback, URL video, tiêu chí)
- Gọi API POST /save_evaluation với dữ liệu
- Hiển thị kết quả vào bảng và thông báo
- Xử lý lỗi nếu có

2. Backend:
- Nhận request POST với dữ liệu đánh giá
- Tạo bảng evaluations nếu chưa có
- Insert dữ liệu vào database
- Trả về kết quả cho frontend

3. Cấu trúc bảng evaluations:
- id: khóa chính tự tăng
- video_url: URL của video được đánh giá  
- criteria: tiêu chí đánh giá
- score: điểm số
- feedback: phản hồi của người đánh giá
- created_at: thời điểm tạo (tự động)

Như vậy mỗi khi user nhập điểm và feedback, dữ liệu sẽ được lưu vào database để có thể truy xuất và phân tích sau này.

Để thiết kế khoá chính - khoá phụ cho bài toán này, ta cần phân tích mối quan hệ giữa các bảng:

1. Bảng videos (bảng chính):
```python
cursor.execute('''
CREATE TABLE IF NOT EXISTS videos (
    video_id INTEGER PRIMARY KEY AUTOINCREMENT,
    url_video TEXT UNIQUE NOT NULL,  
    transcription TEXT,
    criteria TEXT
)
''')
```

2. Bảng evaluations (bảng chứa đánh giá):
```python
cursor.execute('''
CREATE TABLE IF NOT EXISTS evaluations (
    evaluation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    video_id INTEGER NOT NULL,
    criteria TEXT NOT NULL,
    score INTEGER CHECK(score >= 0 AND score <= 5),
    note TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (video_id) REFERENCES videos(video_id) ON DELETE CASCADE,
    UNIQUE(video_id, criteria) -- Đảm bảo mỗi video chỉ có 1 đánh giá cho mỗi tiêu chí
)
''')
```

3. Cập nhật API endpoint để sử dụng khoá ngoại:

```python
@app.route('/save_evaluation', methods=['POST'])
def save_evaluation():
    try:
        data = request.json
        
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Lấy video_id từ url_video
        cursor.execute('SELECT video_id FROM videos WHERE url_video = ?', 
                      (data['videoUrl'],))
        result = cursor.fetchone()
        
        if not result:
            return jsonify({'error': 'Video not found'}), 404
            
        video_id = result[0]

        # Insert hoặc Update đánh giá
        cursor.execute('''
            INSERT INTO evaluations (video_id, criteria, score, feedback)
            VALUES (?, ?, ?, ?)
            ON CONFLICT(video_id, criteria) 
            DO UPDATE SET 
                score = excluded.score,
                feedback = excluded.feedback,
                created_at = CURRENT_TIMESTAMP
        ''', (video_id, data['criteria'], data['score'], data['feedback']))

        conn.commit()
        return jsonify({'message': 'Evaluation saved successfully'}), 200

    except Exception as e:
        print(f"Error saving evaluation: {e}")
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()
```

4. API để lấy tất cả đánh giá của một video:

```python
@app.route('/get_video_evaluations', methods=['GET'])
def get_video_evaluations():
    url_video = request.args.get('url')
    
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        cursor.execute('''
            SELECT e.criteria, e.score, e.feedback, e.created_at
            FROM evaluations e
            JOIN videos v ON e.video_id = v.video_id
            WHERE v.url_video = ?
            ORDER BY e.created_at DESC
        ''', (url_video,))
        
        evaluations = cursor.fetchall()
        
        if evaluations:
            return jsonify([{
                'criteria': eval[0],
                'score': eval[1],
                'feedback': eval[2],
                'created_at': eval[3]
            } for eval in evaluations])
            
        return jsonify({'message': 'No evaluations found'}), 404

    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()
```

Giải thích thiết kế:

1. Khoá chính (Primary Keys):
- videos.video_id: ID tự tăng của video
- evaluations.evaluation_id: ID tự tăng của đánh giá

2. Khoá phụ (Foreign Keys):
- evaluations.video_id: Tham chiếu đến videos.video_id
- Có ràng buộc ON DELETE CASCADE để xoá các đánh giá khi video bị xoá

3. Ràng buộc khác:
- url_video là UNIQUE để tránh trùng lặp video
- score có CHECK để đảm bảo giá trị từ 0-5
- UNIQUE(video_id, criteria) đảm bảo mỗi video chỉ có 1 đánh giá cho mỗi tiêu chí

4. Các trường thời gian:
- created_at tự động lưu thời điểm tạo/cập nhật đánh giá

Với thiết kế này:
- Đảm bảo tính toàn vẹn dữ liệu
- Dễ dàng truy vấn và thống kê
- Tránh được trùng lặp đánh giá
- Có thể mở rộng thêm các bảng liên quan (ví dụ: users, comments...)

# 2. Bug: Khi save điểm xong thì web bị load lại. 

Mình xem lại code cũ thì thấy
```
ý mình là tại sao đoạn này là đoạn feedback mà lại cần call tới cổng get_video_data
```


Cảm ơn bạn đã làm rõ câu hỏi. Để giải thích tại sao đoạn mã liên quan đến việc gửi feedback lại cần gọi đến cổng `get_video_data`, chúng ta cần xem xét cách mà dữ liệu được xử lý trong ứng dụng của bạn.

### Mối liên hệ giữa feedback và `get_video_data`

1. **Lấy thông tin video**: Khi người dùng nhập feedback và nhấn nút lưu, ứng dụng cần biết thông tin chi tiết về video mà người dùng đang đánh giá. Để làm điều này, nó cần gọi đến API `get_video_data` để lấy thông tin về video đó, bao gồm transcript và các tiêu chí liên quan.

2. **Cập nhật thông tin**: Đoạn mã gọi đến `get_video_data` có thể được sử dụng để lấy lại thông tin mới nhất về video, bao gồm cả các tiêu chí mà người dùng đã đánh giá. Điều này giúp đảm bảo rằng thông tin hiển thị trên giao diện người dùng là chính xác và cập nhật.

3. **Xử lý feedback**: Khi người dùng gửi feedback, ứng dụng có thể cần xác nhận hoặc lấy lại thông tin về video để hiển thị điểm số và lý do liên quan đến tiêu chí mà người dùng đã đánh giá. Việc gọi đến `get_video_data` giúp đảm bảo rằng ứng dụng có tất cả thông tin cần thiết để xử lý feedback một cách chính xác.

### Ví dụ cụ thể

Giả sử bạn có một video mà người dùng đã xem và đánh giá. Khi người dùng nhấn nút lưu feedback, ứng dụng sẽ:

- Gửi yêu cầu đến API `save_evaluation` để lưu feedback vào cơ sở dữ liệu.
- Gọi đến API `get_video_data` để lấy lại thông tin về video đó, bao gồm transcript và các tiêu chí đánh giá.
- Cập nhật giao diện người dùng với thông tin mới nhất về video, bao gồm điểm số và lý do cho các tiêu chí mà người dùng đã đánh giá.

### Tóm lại

Việc gọi đến cổng `get_video_data` trong ngữ cảnh của feedback là cần thiết để đảm bảo rằng ứng dụng có thông tin đầy đủ và chính xác về video mà người dùng đang đánh giá. Điều này giúp cải thiện trải nghiệm người dùng bằng cách cung cấp thông tin cập nhật và chính xác về video và các tiêu chí liên quan.

# 3. okei, thế @script.js đúng như bạn nói. 
TÔI MUỐN UPDATE thêm việc lưu 2 thứ: 
1. là score
2. là note
... các cột data khác liên quan như trong bảng evaluations tham khảo
 về bảng này trong video_database.db
---------
Gửi yêu cầu đến API save_evaluation để lưu video, note vào cơ sở dữ liệu.
Gọi đến API get_video_data để lấy lại thông tin về video đó, bao gồm transcript và các tiêu chí đánh giá.
Cập nhật giao diện người dùng với thông tin mới nhất về video, bao gồm điểm số và note cho các tiêu chí mà người dùng đã đánh giá.


# 4. Thử cách không cần gọi 3000 khi save score, notes như ver deploy1.2
- Đoạn save: score và notes. 
- Khi user nhấn save, bạn sẽ 
gửi POST crul sau 

```bash
curl --location 'http://127.0.0.1:3000/save_evaluation' \
--header 'Content-Type: application/json' \
--data '{
           "url_video": "https://example.com/video1",
           "criteria": "Clarity",
           "score": 4,
           "note": "Clear and concise presentation"
         }'
```
Trong đó: 
- url_video chính là cái user nhập lúc đầu tiên. 
- criteri: là tiêu chí mà đang score, note. 
===========
Tuyệt đối không động đến logic các phần ko liên quan


# 5. tại sao sau khi ấn save thì trang bị load lại??? có phải do 2 api ở cùng 1 file 

# 6. Vẫn chưa fix được. Khôi phục từ đầu xong hỏi Claude3.5 web 

Ngoài api save điểm lên, ta thêm 1 api để kéo điểm đã chấm đó hiển thị trên UI luôn 
=> tổng hiện tại 3 api: get_video, save_score, get_score


**Hiện tại điểm hiển thị trên UI là do hai nguồn:**

1. **Khi Save điểm mới:** 
   - Điểm hiển thị ngay thông qua `displayResults()` trong event listener của nút Save

2. **Khi Load video:** 
   - Hệ thống gọi `loadSavedScores()` để load điểm từ database
   - Sau đó cũng dùng `displayResults()` để hiển thị lên UI

